#### Домашнее задание к занятию "Алгоритмы классификации: линейные методы, логистическая регрессия и SVM"

Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
data.shape

(48842, 15)

In [4]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [5]:
data.describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
count,48842,48842,48842,48842,48842,48842,48842,48842,48842
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,33906,15784,22379,6172,19716,41762,32650,43832,37155


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
# data.isna().sum()

Удалим строки с неизвестными значениями. Явно с помощью isna() это сделать нельзя, поскольку неизвестное значение спрятано под вопросительным знаком (?). Поэтому удалим строки, в которых есть вопросительный знак. На этой операции мы теряем около 5% данных, что вполне допустимо.

In [8]:
data = data.loc[~(data.workclass == '?')]
data = data.loc[~(data.occupation == '?')]
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


Оставляем только те признаки, которые нам пригодятся для обучения модели.

In [9]:
selectedColumns = data.drop(['fnlwgt', 'educational-num', 'capital-gain','capital-loss', 'income'], axis=1)
selectedColumns.head()

# or:
# selectedColumns = data[ ['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender',
#                          'hours-per-week', 'native-country'] ]

,age,workclass,education,marital-status,occupation,relationship,race,gender,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States
5,34,Private,10th,Never-married,Other-service,Not-in-family,White,Male,30,United-States


In [10]:
selectedColumns.head()

,age,workclass,education,marital-status,occupation,relationship,race,gender,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States
5,34,Private,10th,Never-married,Other-service,Not-in-family,White,Male,30,United-States


Переведем категориальные переменные (т.е. все, кроме age	и hours-per-week) в 0 и 1 через pd.get_dummies().
Это аналог OneHotEncoder.

In [11]:
X = pd.get_dummies( selectedColumns, columns = [ 'workclass', 'education', 'marital-status', 'occupation', 'relationship',
                                                'race', 'gender', 'native-country' ] )
X.head()

,age,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,50,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,40,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,30,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Целевая переменная (столбец income) снова является категориальной. Переведем значения столбца в числа, оставив один столбец

In [12]:
le = LabelEncoder()
le.fit(data['income'])

LabelEncoder()

In [13]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

Посмотрим, как происходит расшифровка столбца income

In [14]:
le.transform(['<=50K', '<=50K', '>50K', '>50K'])

array([0, 0, 1, 1], dtype=int64)

In [15]:
le.transform(data['income'])

array([0, 0, 1, ..., 0, 0, 1])

Записываем в переменную y преобразованный столбец income

In [16]:
y = pd.Series(le.transform(data['income']))
y.head()

# В лекции было так. Есть ли разница?
# y = pd.Series(data = le.transform(data['income']))

0    0
1    0
2    1
3    1
4    0
dtype: int32

#### Logistic Regression

In [17]:
model = LogisticRegression()

Обучаем модель

In [18]:
model.fit(X, y)

C:\Users\gogol\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [19]:
model.predict(X)

array([0, 0, 0, ..., 0, 0, 1])

In [20]:
predictions = model.predict_proba(X)
predictions[:5]

array([[0.99594157, 0.00405843],
       [0.82522253, 0.17477747],
       [0.64434014, 0.35565986],
       [0.76058809, 0.23941191],
       [0.99470504, 0.00529496]])

In [21]:
model.score(X, y)

0.8302956574631243

#### SVM (SVC)

https://scikit-learn.org/stable/modules/svm.html#svm-classification

In [22]:
model_svm = svm.SVC()

Обучаем модель

In [23]:
model_svm.fit(X, y)

SVC()

In [24]:
model_svm.predict(X)

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
model_svm.score(X, y)

0.8213455564486347